In [1]:
import pandas as pd
import sqlite3
from matplotlib import pyplot as plt

In [2]:
conn = sqlite3.connect("../Resources/gdp_olympic.sqlite")

## Winter

In [3]:
# Winter Table
winter_df = pd.read_sql('SELECT year, country_code, medal FROM winter', conn)
winter_df.head()

,year,country_code,medal
0,1924,FRA,Bronze
1,1924,FRA,Bronze
2,1924,FRA,Bronze
3,1924,FRA,Bronze
4,1924,SUI,Gold


## GDP

In [4]:
# Country GDPs
wdi_df = pd.read_sql('SELECT country_name, country_code, year, gdp_per_cap FROM wdi', conn)
wdi_df.head()

,country_name,country_code,year,gdp_per_cap
0,Afghanistan,AFG,1960,59.77
1,Afghanistan,AFG,1961,59.86
2,Afghanistan,AFG,1962,58.46
3,Afghanistan,AFG,1963,78.71
4,Afghanistan,AFG,1964,82.1


## Merge: Games & GDP

In [7]:
# Merge with country GDPs
df = pd.merge(winter_df, wdi_df, left_on=['country_code', 'year'], right_on=['country_code', 'year'])
df.head()

,year,country_code,medal,country_name,gdp_per_cap
0,1960,RUS,Bronze,Russian Federation,
1,1960,RUS,Bronze,Russian Federation,
2,1960,RUS,Bronze,Russian Federation,
3,1960,RUS,Bronze,Russian Federation,
4,1960,RUS,Bronze,Russian Federation,


### Count of Medals

In [8]:
# Count of Medals
df2 = df.groupby(['year', 'country_name', 'medal']).count()
df2 = df2.reset_index()
df2.head()

,year,country_name,medal,country_code,gdp_per_cap
0,1960,Austria,Bronze,3,3
1,1960,Austria,Gold,1,1
2,1960,Austria,Silver,2,2
3,1960,Canada,Bronze,1,1
4,1960,Canada,Gold,3,3


In [9]:
df2 = df2.iloc[:, 0:4]

In [10]:
df2 = df2.rename(columns={'country_code':'medal_count'})
df2.head()

,year,country_name,medal,medal_count
0,1960,Austria,Bronze,3
1,1960,Austria,Gold,1
2,1960,Austria,Silver,2
3,1960,Canada,Bronze,1
4,1960,Canada,Gold,3


In [11]:
medals_df = pd.pivot_table(df2, values='medal_count', index=['year', 'country_name'], columns='medal', fill_value=0)
medals_df = medals_df.reset_index()
medals_df.head()

medal,year,country_name,Bronze,Gold,Silver
0,1960,Austria,3,1,2
1,1960,Canada,1,3,17
2,1960,Czech Republic,0,0,1
3,1960,Eswatini,0,2,0
4,1960,Finland,5,5,3


### GDP per year

In [13]:
# GDP per year
df3 = df.groupby(['year', 'country_code']).max()
df3 = df3.reset_index()
df3 = df3.drop(columns=['medal'])
df3.head()

,year,country_code,country_name,gdp_per_cap
0,1960,AUT,Austria,935.46
1,1960,CAN,Canada,
2,1960,CZE,Czech Republic,
3,1960,FIN,Finland,1179.35
4,1960,FRA,France,1343.83


In [14]:
final_df = pd.merge(medals_df, df3, left_on=['country_name', 'year'], right_on=['country_name', 'year'])
final_df.head()

,year,country_name,Bronze,Gold,Silver,country_code,gdp_per_cap
0,1960,Austria,3,1,2,AUT,935.46
1,1960,Canada,1,3,17,CAN,
2,1960,Czech Republic,0,0,1,CZE,
3,1960,Eswatini,0,2,0,SUI,104.21
4,1960,Finland,5,5,3,FIN,1179.35


In [15]:
# Reorder columns
final_df = final_df[['year','country_name','country_code','gdp_per_cap', 'Gold', 'Silver', 'Bronze']]
final_df.head()

,year,country_name,country_code,gdp_per_cap,Gold,Silver,Bronze
0,1960,Austria,AUT,935.46,1,2,3
1,1960,Canada,CAN,,3,17,1
2,1960,Czech Republic,CZE,,0,1,0
3,1960,Eswatini,SUI,104.21,2,0,0
4,1960,Finland,FIN,1179.35,5,3,5


In [16]:
# Rename columns
final_df = final_df.rename(columns={'country_name':'country',
                         'gdp_per_cap':'gdp',
                         'Gold':'gold',
                         'Silver':'silver',
                         'Bronze':'bronze'
                        })
final_df.head()

,year,country,country_code,gdp,gold,silver,bronze
0,1960,Austria,AUT,935.46,1,2,3
1,1960,Canada,CAN,,3,17,1
2,1960,Czech Republic,CZE,,0,1,0
3,1960,Eswatini,SUI,104.21,2,0,0
4,1960,Finland,FIN,1179.35,5,3,5


## Export data to dictionary

In [17]:
gdp_medals = final_df.to_dict(orient='records')

In [18]:
gdp_medals

[{'year': 1960,
  'country': 'Austria',
  'country_code': 'AUT',
  'gdp': 935.46,
  'gold': 1,
  'silver': 2,
  'bronze': 3},
 {'year': 1960,
  'country': 'Canada',
  'country_code': 'CAN',
  'gdp': '',
  'gold': 3,
  'silver': 17,
  'bronze': 1},
 {'year': 1960,
  'country': 'Czech Republic',
  'country_code': 'CZE',
  'gdp': '',
  'gold': 0,
  'silver': 1,
  'bronze': 0},
 {'year': 1960,
  'country': 'Eswatini',
  'country_code': 'SUI',
  'gdp': 104.21,
  'gold': 2,
  'silver': 0,
  'bronze': 0},
 {'year': 1960,
  'country': 'Finland',
  'country_code': 'FIN',
  'gdp': 1179.35,
  'gold': 5,
  'silver': 3,
  'bronze': 5},
 {'year': 1960,
  'country': 'France',
  'country_code': 'FRA',
  'gdp': 1343.83,
  'gold': 1,
  'silver': 0,
  'bronze': 2},
 {'year': 1960,
  'country': 'Italy',
  'country_code': 'ITA',
  'gdp': 804.49,
  'gold': 0,
  'silver': 0,
  'bronze': 1},
 {'year': 1960,
  'country': 'Netherlands',
  'country_code': 'NED',
  'gdp': 1068.78,
  'gold': 0,
  'silver': 1,
  'b